In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from google.colab import files
#adicionando novas bibliotecas
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [2]:
files.upload()

Output hidden; open in https://colab.research.google.com to view.

In [4]:
tweets = pd.read_csv("cyberbullying_tweets.csv")
tweets.head()

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was cra...",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImA...,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red ve...,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, b...",not_cyberbullying
4,@RudhoeEnglish This is an ISIS account pretend...,not_cyberbullying


In [7]:
#Encoder para as classes do dataset (spam - not-spam)
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(tweets["cyberbullying_type"])
print(y)

[3 3 3 ... 1 1 1]


In [8]:
mensagens = tweets["tweet_text"].values
x_train, x_test, y_train, y_test = train_test_split(mensagens, y, test_size=0.3)

In [9]:
#Tokenização
token = Tokenizer(num_words=1000)
token.fit_on_texts(x_train)

x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

print(x_train)

Output hidden; open in https://colab.research.google.com to view.

In [10]:
x_train = pad_sequences(x_train, padding='post', maxlen=500)
x_test = pad_sequences(x_test, padding='post', maxlen=500)
print(x_train)

[[  6 134  25 ...   0   0   0]
 [ 64 121  10 ...   0   0   0]
 [  1 540   2 ...   0   0   0]
 ...
 [  9  14 188 ...   0   0   0]
 [ 87 134  12 ...   0   0   0]
 [  5  70  20 ...   0   0   0]]


In [11]:
#criando a rede profunda
modelo = Sequential()
modelo.add(Embedding(input_dim=len(token.word_index), output_dim=50, input_length=500))
modelo.add(Flatten())
modelo.add(Dense(units=10, activation="relu"))
modelo.add(Dropout(0.1))
modelo.add(Dense(units=1, activation="sigmoid"))

In [12]:
#Compilando o modelo, passando parâmetros loss, optimizer e metrics
modelo.compile(loss="mean_squared_error", optimizer="adam", metrics=["accuracy"])

In [13]:
#Visualizando o modelo
modelo.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 50)           2588850   
                                                                 
 flatten (Flatten)           (None, 25000)             0         
                                                                 
 dense (Dense)               (None, 10)                250010    
                                                                 
 dropout (Dropout)           (None, 10)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 2838871 (10.83 MB)
Trainable params: 2838871 (10.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
#treinamento da rede
modelo.fit(x_train, y_train, epochs=20, verbose=True, batch_size=10,
           validation_data=(x_test, y_test))

Epoch 1/20
3339/3339 [==============================] - 125s 37ms/step - loss: 5.1693 - accuracy: 0.1657 - val_loss: 5.1443 - val_accuracy: 0.1697
Epoch 2/20
3339/3339 [==============================] - 116s 35ms/step - loss: 5.1676 - accuracy: 0.1657 - val_loss: 5.1443 - val_accuracy: 0.1697
Epoch 3/20
3339/3339 [==============================] - 115s 34ms/step - loss: 5.1676 - accuracy: 0.1657 - val_loss: 5.1443 - val_accuracy: 0.1697
Epoch 4/20
3339/3339 [==============================] - 115s 34ms/step - loss: 5.1677 - accuracy: 0.1657 - val_loss: 5.1443 - val_accuracy: 0.1697
Epoch 5/20
3339/3339 [==============================] - 119s 36ms/step - loss: 5.1677 - accuracy: 0.1657 - val_loss: 5.1443 - val_accuracy: 0.1697
Epoch 6/20
3339/3339 [==============================] - 119s 36ms/step - loss: 5.1677 - accuracy: 0.1657 - val_loss: 5.1443 - val_accuracy: 0.1697
Epoch 7/20
3339/3339 [==============================] - 123s 37ms/step - loss: 5.1676 - accuracy: 0.1657 - val_loss: 5

In [ ]:
#Matriz de confusão
previsao = modelo.predict(x_test)

53/53 [==============================] - 0s 5ms/step


In [ ]:
print(previsao)

[[2.2603651e-06]
 [7.0313894e-11]
 [8.9349567e-07]
 ...
 [8.4778442e-09]
 [1.0000000e+00]
 [5.3008384e-09]]


In [ ]:
previsao_convertido = (previsao > 0.5)
matriz_confusao = confusion_matrix(y_test, previsao_convertido)
print(matriz_confusao)

[[1452    9]
 [  15  196]]
